In [1]:
import psycopg2
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
from plotly.offline import plot 
import plotly.graph_objects as go
import warnings
import datetime
warnings.filterwarnings('ignore')

# import q_credentials.db_secmaster_cloud_cred as db_secmaster_cred
import q_credentials.db_secmaster_cred as db_secmaster_cred
import q_credentials.db_indicator_cred as db_indicator_cred


In [2]:
# connect to our securities_master database
conn_secmaster = psycopg2.connect(host=db_secmaster_cred.dbHost , database=db_secmaster_cred.dbName, user=db_secmaster_cred.dbUser, password=db_secmaster_cred.dbPWD)

In [3]:
conn_indicator = psycopg2.connect(host=db_indicator_cred.dbHost , database=db_indicator_cred.dbName, user=db_indicator_cred.dbUser, password=db_indicator_cred.dbPWD)

In [4]:
sql="select * from indicator"
ind_list=list(pd.read_sql(sql,con=conn_indicator)['name'])
ind_list

['candle_1', 'candle_2', 'candle_3', 'anomaly', 'level']

In [243]:
symbol_id='TSI'#'BOM500114'
start_date=datetime.datetime(2016,1,1).strftime("%Y-%m-%d")
indicator_name = 'candle_1'

In [244]:
df_all_ind=pd.DataFrame()
for ind in ind_list:
    print(ind)
    sql="select d.date_price as date, d.value from w_data d join symbol s on d.symbol_id = s.id join indicator i on i.id=d.indicator_id where s.ticker='%s' and i.name = '%s' and d.date_price > '%s' order by date" %(symbol_id, ind, start_date)
    df_indicator=pd.read_sql(sql,con=conn_indicator)
    df_indicator.set_index('date',inplace=True)
    df_indicator=pd.concat([df_indicator.drop(['value'], axis=1), df_indicator['value'].apply(pd.Series)], axis=1)
    df_indicator.columns=[ind+"_"+col for col in df_indicator.columns]
    if df_all_ind.empty:
#         df_all_ind=pd.merge(left=df_price, right=df_indicator,on='date')
        df_all_ind=df_indicator
    else:
        df_all_ind=pd.merge(left=df_all_ind, right=df_indicator,on='date')

candle_1
candle_2
candle_3
anomaly
level


In [245]:
# df_all_ind.head()

In [246]:
df_all_ind.rename(columns={'anomaly_close':'close','anomaly_low':'low','anomaly_high':'high','anomaly_open':'open','anomaly_volume':'volume'},inplace=True)

In [247]:
pd.options.display.max_columns = None

In [248]:
# df.iloc[-1]

In [249]:
# [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.loc[df.index[-1]]['level_support']]

In [250]:
def level_plot(df):
    try:
        support_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_support']]
    except:
        support_ls =[]
    try:
        resistance_ls = [[ls[0],ls[1],datetime.datetime.strptime(ls[2],'%Y-%m-%d %H:%M:%S'),ls[3]] for ls in df.iloc[-1]['level_resistance']]
    except:
        resistance_ls =[]
    end_dt=df.index[-1]
    res_plot_ls=[]
    sup_plot_ls=[]
    for res in resistance_ls[:5]:
        res_plot_ls.append(dict(x0=res[2],x1=end_dt,y0=res[0],y1=res[1],yref='y1',opacity=.2,fillcolor='Red',line=dict(color="black",width=1)))
    for sup in support_ls[:5]:
        sup_plot_ls.append(dict(x0=sup[2],x1=end_dt,y0=sup[0],y1=sup[1],yref='y1',opacity=.2,fillcolor='green',line=dict(color="black",width=1)))
    return (res_plot_ls+sup_plot_ls)

In [251]:
df.columns

Index(['candle_1_high', 'candle_1_level', 'candle_1_low',
       'candle_1_pattern_name', 'candle_2_high', 'candle_2_level',
       'candle_2_low', 'candle_2_pattern_name', 'candle_3_high',
       'candle_3_level', 'candle_3_low', 'candle_3_pattern_name',
       'anomaly_atr', 'anomaly_bear', 'anomaly_body_size',
       'anomaly_body_size_dev', 'anomaly_body_size_ema', 'anomaly_bull',
       'close', 'anomaly_gap', 'anomaly_gap_dev', 'anomaly_gap_ema',
       'anomaly_gap_signal', 'high', 'low', 'open', 'anomaly_vol_anomaly',
       'anomaly_vol_anomaly_2', 'anomaly_vol_dev', 'anomaly_vol_ema', 'volume',
       'anomaly_wick_bear', 'anomaly_wick_bear_dev', 'anomaly_wick_bear_ema',
       'anomaly_wick_bear_signal', 'anomaly_wick_bull',
       'anomaly_wick_bull_dev', 'anomaly_wick_bull_ema',
       'anomaly_wick_bull_signal', 'level_level',
       'level_nearest_resistance_atr_range',
       'level_nearest_resistance_duration', 'level_nearest_resistance_range',
       'level_nearest_su

In [252]:
# candle1_plot_ls=[]
# for i,row in df.tail(10).iterrows():
#     if(row['candle_1_pattern_name']):
# #         print(row['candle_1_pattern_name'])
#         print(pd.to_datetime(i), (i+datetime.timedelta(days=1)))
#         candle1_plot_ls.append(dict(x0=i,x1=(i+datetime.timedelta(days=2)),y0=row['high'],y1=row['low'],yref='y1',opacity=.2,fillcolor='blue',line=dict(color="black",width=1)))

In [253]:
# df=df_all_ind.set_index('date')
df=df_all_ind
interested_feature='anomaly_vol_anomaly'

In [254]:
df_candle_1=df
df_candle_1=df_candle_1[df_candle_1['candle_1_pattern_name']!='']
df_candle_1=df_candle_1[~df_candle_1['candle_1_pattern_name'].isin(['long', 'short'])]
# df_candle_1=df_candle_1[df_candle_1['candle_1_pattern_name'].isin(['HAMMER', 'INV_HAMMER'])]
df_candle_1['pattern']='1'

df_candle_2=df
df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name']!='']
# df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name'].isin(['BULLISH ENGULFING', 'BEARISH ENGULFING'])]
df_candle_2=df_candle_2[df_candle_2['candle_2_pattern_name'].isin(['BEARISH (Dark Cloud Cover)', 'BULLISH (Piercing Pattern)'])]
df_candle_2['pattern']='2'

df_candle_3=df
df_candle_3=df_candle_3[df_candle_3['candle_3_pattern_name']!='']
df_candle_3['pattern']='3'

In [255]:
df.columns

Index(['candle_1_high', 'candle_1_level', 'candle_1_low',
       'candle_1_pattern_name', 'candle_2_high', 'candle_2_level',
       'candle_2_low', 'candle_2_pattern_name', 'candle_3_high',
       'candle_3_level', 'candle_3_low', 'candle_3_pattern_name',
       'anomaly_atr', 'anomaly_bear', 'anomaly_body_size',
       'anomaly_body_size_dev', 'anomaly_body_size_ema', 'anomaly_bull',
       'close', 'anomaly_gap', 'anomaly_gap_dev', 'anomaly_gap_ema',
       'anomaly_gap_signal', 'high', 'low', 'open', 'anomaly_vol_anomaly',
       'anomaly_vol_anomaly_2', 'anomaly_vol_dev', 'anomaly_vol_ema', 'volume',
       'anomaly_wick_bear', 'anomaly_wick_bear_dev', 'anomaly_wick_bear_ema',
       'anomaly_wick_bear_signal', 'anomaly_wick_bull',
       'anomaly_wick_bull_dev', 'anomaly_wick_bull_ema',
       'anomaly_wick_bull_signal', 'level_level',
       'level_nearest_resistance_atr_range',
       'level_nearest_resistance_duration', 'level_nearest_resistance_range',
       'level_resistance

In [262]:
# Plotting
data = [ dict(
    type = 'candlestick',
    open = df.open,
    high = df.high,
    low = df.low,
    close = df.close,
    x = df.index,
    yaxis = 'y1',
    name = 'price'
)]


# data.append( dict( x=df.index, y=df.volume,                         
#                          marker=dict( color='blue' ),
#                          type='bar', yaxis='y2', name='Volume'))

# data.append( dict( x=df.index, y=df[interested_feature],                         
#                          marker=dict( color='red' ),
#                          type='scatter', yaxis='y3', name=interested_feature))

data.append( dict( x=df.index, y=df['anomaly_wick_bear_signal'],                         
                         marker=dict( color='blue' ),
                         type='scatter', yaxis='y2', name='anomaly_wick_bear_signal'))

data.append( dict( x=df.index, y=df['anomaly_wick_bull_signal'],                         
                         marker=dict( color='red' ),
                         type='scatter', yaxis='y3', name='anomaly_wick_bull_signal'))

# data.append( dict( x=df_candle_.index, y=df_candle_1['high'],
#                          text=df_candle_1['candle_1_pattern_name'],
#                          mode="markers",
# #                          mode="markers+text",textfont_size=15,textposition="top center",
#                          marker=dict( color='blue' ),
#                          type='scatter', yaxis='y1', name='candle_patter_1'))

data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
                         text=df_candle_2['candle_2_pattern_name'],
                         mode="markers",
                         marker=dict( color='blue' ),
                         type='scatter', yaxis='y1', name='candle_patter_2'))

# data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
#                          text=df_candle_2['candle_2_pattern_name'],
# #                          mode="markers+text",textfont_size=15,textposition="top center",
#                          marker=dict( color='red' ),
#                          type='scatter', yaxis='y1', name='candle_patter_2'))


# data.append( dict( x=df_candle_2.index, y=df_candle_2['high'],
#                          text=df_candle_2['candle_2_pattern_name'],
#                          mode="markers+text",textfont_size=15,textposition="top center",
#                          marker=dict( color='red' ),
#                          type='scatter', yaxis='y1', name='candle_patter_2'))


# data.append( dict( x=df_candle_3.index, y=df_candle_3['high'],
#                          text=df_candle_3['pattern'],
#                          mode="markers+text",textfont_size=15,textposition="top center",
#                          marker=dict( color='red' ),
#                          type='scatter', yaxis='y1', name='candle_patter_3'))



layout=dict()    
layout['xaxis'] = dict( rangeslider = dict( visible = False ),autorange=True,fixedrange=False,visible=False,type='category')#type='category',
layout['yaxis'] = dict( domain = [0.2, 1],autorange = True,fixedrange=False)
layout['yaxis2'] = dict( domain = [0.0, 0.1],autorange = True,fixedrange=False)
layout['yaxis3'] = dict( domain = [0.1, 0.2],autorange = True,fixedrange=False)
# layout['hovermode']  = 'x'
layout['xaxis']['showspikes']=True
layout['xaxis']['spikemode']  = 'across'
layout['xaxis']['spikesnap'] = 'cursor'
# layout['shapes'] = [dict(x0=start_dt, x1=end_dt, y0=3057.9, y1=3060.85, yref='y1',opacity=.2,line=dict(color="Red",width=1))]#fillcolor='green',line_width=2
layout['shapes'] = level_plot(df)
# layout['shapes'] = candle1_plot_ls

# df.to_csv("stock_price_indicator.csv",index=True)

fig = dict( data=data, layout=layout )

In [263]:
# for i in range(4):
#     figure.add_annotation(x=i,
#             y=5,
#             xref="x",
#             yref="y",
#             text="max="+str(i),
#             showarrow=True
#                      )

In [264]:
figure=go.FigureWidget(data=data, layout=layout)
def zoom(layout, x_range):
    print(x_range)
    in_view = df.iloc[int(figure.layout.xaxis.range[0]):int(figure.layout.xaxis.range[1])]
    figure.layout.yaxis2.range = [in_view.volume.min(), in_view.volume.max()]
    figure.layout.yaxis3.range = [in_view[interested_feature].min(), in_view[interested_feature].max()]
figure.layout.on_change(zoom, 'xaxis.range')
figure.update_layout(margin=dict(l=20, r=10),paper_bgcolor="LightSteelBlue",width=2200, height=1000)

FigureWidget({
    'data': [{'close': array([5.32  , 5.28  , 5.31  , ..., 5.64  , 5.71  , 5.7056]),
          …

In [205]:
figure=go.FigureWidget(data=data, layout=layout)
def zoom(layout, x_range):
    print(x_range)
    in_view = df.iloc[int(figure.layout.xaxis.range[0]):int(figure.layout.xaxis.range[1])]
    figure.layout.yaxis2.range = [in_view.volume.min(), in_view.volume.max()]
    figure.layout.yaxis3.range = [in_view[interested_feature].min(), in_view[interested_feature].max()]
figure.layout.on_change(zoom, 'xaxis.range')
figure.update_layout(margin=dict(l=20, r=10),paper_bgcolor="LightSteelBlue",width=1200, height=800)

FigureWidget({
    'data': [{'close': array([ 43.75  ,  44.2725,  44.615 ,  42.8775,  40.125 ,  39.1025,  43.1…